In [1]:
# Load libraries
# library(DiffBind)
library(BiocParallel)
library(dplyr)
library(TxDb.Athaliana.BioMart.plantsmart51)
# library(ChIPseeker)

library(chromVAR)
library(motifmatchr)
library(BiocParallel)
register(MulticoreParam(50)) # Use 50 cores
# Set options
options(stringsAsFactors = F)
Fold_cutoff <- 1
FDR_cutoff <- 0.05


Warning message:
“package ‘BiocParallel’ was built under R version 4.3.2”

Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: GenomicFeatures

Warning message:
“package ‘GenomicFeatures’ was built under R version 4.3.2”
Loading required package: BiocGenerics

Warning message:
“package ‘BiocGenerics’ was built under R version 4.3.2”

Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, 

In [2]:
map_id_name <- read.csv('../gene_table_and_other_scripts/bioMartR_Ath_all_genes_info_googled.csv')
map_id_name <- map_id_name %>% dplyr::rename('gene_name' = 'external_gene_name', 'gene_id' = 'ensembl_gene_id')
sample_n(map_id_name, 2)

gene_id,gene_name
<chr>,<chr>
AT2G07768,AT2G07768
AT3G12835,AT3G12835


In [3]:
source('../gene_table_and_other_scripts/theme_Publication.R')

Registering fonts with R

aakar already registered with pdfFont().

Abyssinica SIL already registered with pdfFont().

Andale Mono already registered with pdfFont().

Ani already registered with pdfFont().

AnjaliOldLipi already registered with pdfFont().

Arial already registered with pdfFont().

Arial Black already registered with pdfFont().

Chandas already registered with pdfFont().

Comic Sans MS already registered with pdfFont().

Courier New already registered with pdfFont().

DejaVu Sans already registered with pdfFont().

DejaVu Sans Mono already registered with pdfFont().

DejaVu Serif already registered with pdfFont().

Dhurjati already registered with pdfFont().

Droid Sans Fallback already registered with pdfFont().

Dyuthi already registered with pdfFont().

FreeMono already registered with pdfFont().

FreeSans already registered with pdfFont().

FreeSerif already registered with pdfFont().

Gargi already registered with pdfFont().

Garuda already registered with pdfFont(

https://greenleaflab.github.io/chromVAR/articles/Introduction.html#loading-the-package

In [4]:
library("ggplot2")

In [5]:
library(BSgenome.Athaliana.TAIR.TAIR9)

Loading required package: BSgenome

Warning message:
“package ‘BSgenome’ was built under R version 4.3.2”
Loading required package: Biostrings

Warning message:
“package ‘Biostrings’ was built under R version 4.3.2”
Loading required package: XVector

Warning message:
“package ‘XVector’ was built under R version 4.3.2”

Attaching package: ‘Biostrings’


The following object is masked from ‘package:base’:

    strsplit


Loading required package: BiocIO

Warning message:
“package ‘BiocIO’ was built under R version 4.3.2”
Loading required package: rtracklayer

Warning message:
“package ‘rtracklayer’ was built under R version 4.3.2”

Attaching package: ‘rtracklayer’


The following object is masked from ‘package:BiocIO’:

    FileForFormat




In [6]:
source('../gene_table_and_other_scripts/theme_Publication.R')

aakar already registered with pdfFont().

Abyssinica SIL already registered with pdfFont().

Andale Mono already registered with pdfFont().

Ani already registered with pdfFont().

AnjaliOldLipi already registered with pdfFont().

Arial already registered with pdfFont().

Arial Black already registered with pdfFont().

Chandas already registered with pdfFont().

Comic Sans MS already registered with pdfFont().

Courier New already registered with pdfFont().

DejaVu Sans already registered with pdfFont().

DejaVu Sans Mono already registered with pdfFont().

DejaVu Serif already registered with pdfFont().

Dhurjati already registered with pdfFont().

Droid Sans Fallback already registered with pdfFont().

Dyuthi already registered with pdfFont().

FreeMono already registered with pdfFont().

FreeSans already registered with pdfFont().

FreeSerif already registered with pdfFont().

Gargi already registered with pdfFont().

Garuda already registered with pdfFont().

Georgia already regist

# start with my own data

In [7]:
library(GenomicAlignments)

Warning message:
“package ‘GenomicAlignments’ was built under R version 4.3.2”
Loading required package: SummarizedExperiment

Warning message:
“package ‘SummarizedExperiment’ was built under R version 4.3.2”
Loading required package: MatrixGenerics

Warning message:
“package ‘MatrixGenerics’ was built under R version 4.3.2”
Loading required package: matrixStats


Attaching package: ‘matrixStats’


The following objects are masked from ‘package:Biobase’:

    anyMissing, rowMedians


The following object is masked from ‘package:dplyr’:

    count



Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, 

In [8]:
bamReads_DM1_DM2d = list.files("/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam/", pattern = "bam$", full.names = T)
bamReads_DM6_DM7 = list.files("/media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/4_intersect_bam/", pattern = "bam$", full.names = T)

In [9]:
# concat bamReads_DM1_DM2d and bamReads_DM6_DM7
bamReads <- c(bamReads_DM1_DM2d, bamReads_DM6_DM7)
bamReads

[1] "/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam//C12A_clean_filtered.bam"
 [2] "/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam//C12B_clean_filtered.bam"
 [3] "/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam//C12C_clean_filtered.bam"
 [4] "/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam//DM1_clean_filtered.bam" 
 [5] "/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam//DM2_clean_filtered.bam" 
 [6] "/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam//DM3_clean_filtered.bam" 
 [7] "/media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/4_intersect_bam//C1_L3_clean_filtered.bam"    
 [8] "/media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/4_intersect_bam//C2_L3_clean_filtered.bam"    
 [9] "/media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/4_intersect_bam//C3_L3_clean_filtered.bam"    
[10] "/media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/4_intersect_bam//OK1_L3_clean_filtered.bam"   
[11] "/media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/4_intersect_bam//OK2_L3_clean_filtered.bam"   
[12] "/media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/4_intersect_bam//OK3_L3_clean_filtered.bam"

In [10]:
Peaks_DM1_DM2d = list.files("/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/5_macs2peaks_simple/", pattern = 'narrowPeak$', full.names = T)
Peaks_DM6_DM7 = list.files("/media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/5_macs2peaks_simple/", pattern = 'narrowPeak$', full.names = T)
#concat Peaks_DM1_DM2d and Peaks_DM6_DM7
Peaks <- c(Peaks_DM1_DM2d, Peaks_DM6_DM7)
Peaks

[1] "/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/5_macs2peaks_simple//C12A_clean_filtered_peaks.narrowPeak"
 [2] "/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/5_macs2peaks_simple//C12B_clean_filtered_peaks.narrowPeak"
 [3] "/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/5_macs2peaks_simple//C12C_clean_filtered_peaks.narrowPeak"
 [4] "/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/5_macs2peaks_simple//DM1_clean_filtered_peaks.narrowPeak" 
 [5] "/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/5_macs2peaks_simple//DM2_clean_filtered_peaks.narrowPeak" 
 [6] "/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/5_macs2peaks_simple//DM3_clean_filtered_peaks.narrowPeak" 
 [7] "/media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/5_macs2peaks_simple//C1_L3_clean_filtered_peaks.narrowPeak"    
 [8] "/media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/5_macs2peaks_simple//C2_L3_clean_filtered_peaks.narrowPeak"    
 [9] "/media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/5_macs2peaks_simple//C3_L3_clean_filtered_peaks.narrowPeak"    
[10] "/media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/5_macs2peaks_simple//OK1_L3_clean_filtered_peaks.narrowPeak"   
[11] "/media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/5_macs2peaks_simple//OK2_L3_clean_filtered_peaks.narrowPeak"   
[12] "/media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/5_macs2peaks_simple//OK3_L3_clean_filtered_peaks.narrowPeak"

In [11]:
# Create a sample info data frame
sample_info <- data.frame(
  SampleID = c('C12_1', 'C12_2', 'C12_3', 'DM1-DM2d_1', 'DM1-DM2d_2', 'DM1-DM2d_3',  'C14_1', 'C14_2', 'C14_3', 'DM6-DM7_1', 'DM6-DM7_2', 'DM6-DM7_3'),
    Condition = c('Col-0_12DAS', 'Col-0_12DAS', 'Col-0_12DAS', 'DM1-DM2d', 'DM1-DM2d', 'DM1-DM2d', 'Col-0_14DAS', 'Col-0_14DAS', 'Col-0_14DAS', 'DM6-DM7', 'DM6-DM7', 'DM6-DM7'),
    DAS = c('12DAS', '12DAS', '12DAS', '12DAS', '12DAS', '12DAS',
            '14DAS', '14DAS', '14DAS', '14DAS', '14DAS', '14DAS'),
  Summary_Condition = rep(c('COL', 'DM', 'COL', 'DM'), each = 3),
  Replicate = rep(1:3, 4),
  bamReads = bamReads,
  Peaks = Peaks,
  PeakCaller = "narrow",
  stringsAsFactors = F
)


In [12]:
sample_info

SampleID,Condition,DAS,Summary_Condition,Replicate,bamReads,Peaks,PeakCaller
<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>
C12_1,Col-0_12DAS,12DAS,COL,1,/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam//C12A_clean_filtered.bam,/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/5_macs2peaks_simple//C12A_clean_filtered_peaks.narrowPeak,narrow
C12_2,Col-0_12DAS,12DAS,COL,2,/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam//C12B_clean_filtered.bam,/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/5_macs2peaks_simple//C12B_clean_filtered_peaks.narrowPeak,narrow
C12_3,Col-0_12DAS,12DAS,COL,3,/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam//C12C_clean_filtered.bam,/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/5_macs2peaks_simple//C12C_clean_filtered_peaks.narrowPeak,narrow
DM1-DM2d_1,DM1-DM2d,12DAS,DM,1,/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam//DM1_clean_filtered.bam,/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/5_macs2peaks_simple//DM1_clean_filtered_peaks.narrowPeak,narrow
DM1-DM2d_2,DM1-DM2d,12DAS,DM,2,/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam//DM2_clean_filtered.bam,/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/5_macs2peaks_simple//DM2_clean_filtered_peaks.narrowPeak,narrow
DM1-DM2d_3,DM1-DM2d,12DAS,DM,3,/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam//DM3_clean_filtered.bam,/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/5_macs2peaks_simple//DM3_clean_filtered_peaks.narrowPeak,narrow
C14_1,Col-0_14DAS,14DAS,COL,1,/media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/4_intersect_bam//C1_L3_clean_filtered.bam,/media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/5_macs2peaks_simple//C1_L3_clean_filtered_peaks.narrowPeak,narrow
C14_2,Col-0_14DAS,14DAS,COL,2,/media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/4_intersect_bam//C2_L3_clean_filtered.bam,/media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/5_macs2peaks_simple//C2_L3_clean_filtered_peaks.narrowPeak,narrow
C14_3,Col-0_14DAS,14DAS,COL,3,/media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/4_intersect_bam//C3_L3_clean_filtered.bam,/media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/5_macs2peaks_simple//C3_L3_clean_filtered_peaks.narrowPeak,narrow


In [13]:
# Read in all peak files and combine them
all_peaks <- lapply(Peaks, rtracklayer::import)
combined_peaks <- Reduce(function(x, y) reduce(c(x, y)), all_peaks)
combined_peaks <- unique(combined_peaks)

In [14]:
combined_peaks

GRanges object with 69885 ranges and 0 metadata columns:
          seqnames            ranges strand
             <Rle>         <IRanges>  <Rle>
      [1]        1         1481-1729      *
      [2]        1         2422-3247      *
      [3]        1         4324-5335      *
      [4]        1         5438-5558      *
      [5]        1         6308-6951      *
      ...      ...               ...    ...
  [69881]        5 26971501-26971814      *
  [69882]        5 26972053-26972799      *
  [69883]        5 26973679-26973823      *
  [69884]        5 26974307-26974602      *
  [69885]        5 26975238-26975460      *
  -------
  seqinfo: 5 sequences from an unspecified genome; no seqlengths

In [15]:
library(BiocParallel)
library(GenomicRanges)
library(GenomicAlignments)
library(rtracklayer)

# Assuming combined_peaks and sample_info are already defined

# Register a parallel backend to use several cores
register(MulticoreParam(workers = 20))  # Adjust the number of workers as needed

# Define a function to count reads using summarizeOverlaps
count_reads_in_peaks <- function(bam_file, peaks, sample_id) {
  aln <- readGAlignments(bam_file)
  counts <- summarizeOverlaps(features=peaks, reads=aln, mode="Union",
                              singleEnd=TRUE, ignore.strand=TRUE)
  counts <- assays(counts)$counts
  names(counts) <- sample_id  # Set the names of counts to the corresponding SampleID
  return(counts)
}

# Use bplapply to apply the function in parallel
counts_list <- bplapply(seq_along(bamReads), function(i) {
  count_reads_in_peaks(bamReads[i], combined_peaks, sample_info$SampleID[i])
})
# Combine the list of count vectors into a matrix and ensure the columns match the sample IDs
counts_matrix <- do.call(cbind, counts_list)
colnames(counts_matrix) = sample_info$SampleID
head(counts_matrix)

C12_1,C12_2,C12_3,DM1-DM2d_1,DM1-DM2d_2,DM1-DM2d_3,C14_1,C14_2,C14_3,DM6-DM7_1,DM6-DM7_2,DM6-DM7_3
436,345,277,353,336,358,194,249,197,334,287,219
1442,985,854,781,929,795,698,655,717,911,875,509
1711,1195,925,1162,1367,1210,672,571,709,830,855,678
199,173,126,154,184,166,124,91,86,141,98,141
1390,1006,828,821,970,842,580,523,593,660,704,496
1371,998,933,824,1074,1024,731,694,692,655,867,467


### change Chr names in combined_peaks to match BSgenome, before making se

In [16]:
# Make sure the chromosome names in combined_peaks match those in the BSgenome object
genome <- BSgenome.Athaliana.TAIR.TAIR9

# Map the numeric chromosome names to the BSgenome chromosome names
# The keys in chr_map should exactly match the current seqlevels in combined_peaks
chr_map <- c("1" = "Chr1", "2" = "Chr2", "3" = "Chr3", "4" = "Chr4", "5" = "Chr5")

# Validate the mapping for any NA values
mapped_seqlevels <- chr_map[seqlevels(combined_peaks)]
if (any(is.na(mapped_seqlevels))) {
  stop("One or more seqlevels do not have a corresponding entry in chr_map.")
}

# Now apply the validated mapping to combined_peaks
seqlevels(combined_peaks) <- mapped_seqlevels

In [17]:
combined_peaks

GRanges object with 69885 ranges and 0 metadata columns:
          seqnames            ranges strand
             <Rle>         <IRanges>  <Rle>
      [1]     Chr1         1481-1729      *
      [2]     Chr1         2422-3247      *
      [3]     Chr1         4324-5335      *
      [4]     Chr1         5438-5558      *
      [5]     Chr1         6308-6951      *
      ...      ...               ...    ...
  [69881]     Chr5 26971501-26971814      *
  [69882]     Chr5 26972053-26972799      *
  [69883]     Chr5 26973679-26973823      *
  [69884]     Chr5 26974307-26974602      *
  [69885]     Chr5 26975238-26975460      *
  -------
  seqinfo: 5 sequences from an unspecified genome; no seqlengths

In [18]:
se <- SummarizedExperiment(assays = List(counts = counts_matrix), 
                           rowRanges = combined_peaks, 
                           colData = DataFrame(sample_info))

In [19]:
counts_matrix 

C12_1,C12_2,C12_3,DM1-DM2d_1,DM1-DM2d_2,DM1-DM2d_3,C14_1,C14_2,C14_3,DM6-DM7_1,DM6-DM7_2,DM6-DM7_3
436,345,277,353,336,358,194,249,197,334,287,219
1442,985,854,781,929,795,698,655,717,911,875,509
1711,1195,925,1162,1367,1210,672,571,709,830,855,678
199,173,126,154,184,166,124,91,86,141,98,141
1390,1006,828,821,970,842,580,523,593,660,704,496
1371,998,933,824,1074,1024,731,694,692,655,867,467
1966,1556,1164,1322,1543,1294,677,731,797,992,819,493
422,294,237,305,320,301,128,139,129,171,164,184
595,417,294,406,527,472,220,199,235,290,310,271
2009,1408,1263,759,774,789,886,984,1011,772,824,538


### Getting GC content of peaks

In [20]:
# Assuming combined_peaks is your GRanges object
seqlevels(combined_peaks)


[1] "Chr1" "Chr2" "Chr3" "Chr4" "Chr5"

In [21]:
# Confirm that the seqlevels in combined_peaks now match those in the genome
all(seqlevels(combined_peaks) %in% seqnames(genome))  # should return TRUE

# Proceed to add GC content
se <- addGCBias(se, genome = genome)

[1] TRUE

In [22]:
se <- addGCBias(se, 
       genome = BSgenome.Athaliana.TAIR.TAIR9)
head(rowData(se))

DataFrame with 6 rows and 1 column
       bias
  <numeric>
1  0.353414
2  0.343826
3  0.394269
4  0.479339
5  0.381988
6  0.366319

In [23]:
library(DESeq2)

# Convert SummarizedExperiment to DESeqDataSet
dds <- DESeqDataSet(se, design = ~ Condition)

# Estimate size factors to account for differences in library sizes
dds <- estimateSizeFactors(dds)

# Proceed with normalization
normalized_counts <- counts(dds, normalized=TRUE)

# Replace the raw counts in 'se' with normalized counts
assays(se)$counts <- normalized_counts

# Now 'se' contains normalized counts and you can continue with downstream analysis


Warning message:
“package ‘DESeq2’ was built under R version 4.3.2”
Warning message in DESeqDataSet(se, design = ~Condition):
“some variables in design formula are characters, converting to factors”
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]



https://zhuanlan.zhihu.com/p/623880840

In [24]:
library(TFBSTools)
library(universalmotif)
# Path to your MEME file
meme_file <- "./scan_motif//Ath_TF_binding_motifs_plantTFDB.meme"

# Read the MEME file and convert it to a list of PFMs
Ath_motifs <- read_meme(meme_file)
# Ath_motifs %>% str
Ath_motifs <- convert_motifs(Ath_motifs, class="TFBSTools-PFMatrix")
Ath_motifs_list <- do.call(PFMatrixList,Ath_motifs)
names(Ath_motifs_list)= sapply(Ath_motifs, function(x) slot(x, "name"))
Ath_motifs_list

Warning message:
“package ‘TFBSTools’ was built under R version 4.3.2”
Warning message:
“package ‘universalmotif’ was built under R version 4.3.2”


PFMatrixList of length 619
names(619): AT1G01060 AT1G01250 AT1G01260 ... AT5G67300 AT5G67450 AT5G67580

In [25]:
Ath_motifs_list %>% str

Formal class 'PFMatrixList' [package "TFBSTools"] with 4 slots
  ..@ listData       :List of 619
  .. ..$ AT1G01060:Formal class 'PFMatrix' [package "TFBSTools"] with 7 slots
  .. .. .. ..@ ID           : chr "MP00119"
  .. .. .. ..@ name         : chr "AT1G01060"
  .. .. .. ..@ matrixClass  : chr "Unknown"
  .. .. .. ..@ strand       : chr "*"
  .. .. .. ..@ bg           : Named num [1:4] 0.332 0.168 0.168 0.332
  .. .. .. .. ..- attr(*, "names")= chr [1:4] "A" "C" "G" "T"
  .. .. .. ..@ tags         :List of 1
  .. .. .. .. ..$ eval.string: chr "4.8e-803"
  .. .. .. ..@ profileMatrix: int [1:4, 1:10] 512 1 31 55 0 0 599 0 599 0 ...
  .. .. .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. .. .. ..$ : chr [1:4] "A" "C" "G" "T"
  .. .. .. .. .. ..$ : chr [1:10] "A" "G" "A" "T" ...
  .. ..$ AT1G01250:Formal class 'PFMatrix' [package "TFBSTools"] with 7 slots
  .. .. .. ..@ ID           : chr "MP00120"
  .. .. .. ..@ name         : chr "AT1G01250"
  .. .. .. ..@ matrixClass  : chr "Unkn

In [26]:
library(motifmatchr)
motif_ix <- matchMotifs(Ath_motifs_list, se, 
                        genome = BSgenome.Athaliana.TAIR.TAIR9)

In [27]:
dev <- computeDeviations(object = se, annotations = motif_ix)


ERROR: Error in Matrix::rowSums(object): object 'R_dense_marginsum' not found


In [ ]:
dev

In [ ]:
deviations(dev)[1:3,1:2]

The ‘‘correlation’’ represents the co-accessibility tendency of two TF-binding motifs, whereas ‘‘synergy’’ stands for the frequency by which two TF-binding motifs co-localize in the same accessible peak. 
#Wang, Fu-Xiang, Guan-Dong Shang, Lian-Yu Wu, Zhou-Geng Xu, Xin-Yan Zhao, and Jia-Wei Wang. “Chromatin Accessibility Dynamics and a Hierarchical Transcriptional Regulatory Network Structure for Plant Somatic Embryogenesis.” Developmental Cell 54, no. 6 (September 28, 2020): 742-757.e8. https://doi.org/10.1016/j.devcel.2020.07.003.


### example codes

In [28]:
Cor_matrix <- getAnnotationCorrelation(se, motif_ix[,c(1, 100, 200)])

ERROR: Error in Matrix::rowSums(object): object 'R_dense_marginsum' not found


In [ ]:
Syn_matrix <- getAnnotationSynergy(se, motif_ix[,c(1, 100, 200)])

In [ ]:
library(ggplot2)
library(reshape2)

# Melting the matrix into long format
melted_Cor <- melt(Cor_matrix)
melted_Syn <- melt(Syn_matrix)

In [ ]:
# Creating the heatmap with ggplot2
plot <- ggplot(data = melted_Cor, aes(x=Var1, y=Var2, fill=value)) +
  geom_tile(aes(size = abs(value))) + # Size by the absolute value of synergy level
  scale_fill_gradient2(low = "#5793B3", high = "#CA5351", mid = "white", 
                       midpoint = 0, limit = c(-1,1), space = "Lab", 
                       name="Accessibility\nCorrelation Level") +
  geom_text(data = melted_Syn, aes(label = round(value, 2)), size = 4) + # Add text labels
  theme_minimal() + theme_Arial_8() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1),
        legend.position = "bottom",
        panel.grid.major = element_blank(),
        panel.grid.minor = element_blank(),
        panel.border = element_blank(),
        axis.ticks = element_blank()) + 
  labs(title = "Correlation and synergy between pairs of TFs") +
     scale_x_discrete( expand = c(0, 0)) + scale_y_discrete( expand = c(0, 0)) +

  labs(x = "", y = "", title = "Correlation and synergy between pairs of TF motifs")

plot

### real TF of interest

In [ ]:
# Create a named vector for mapping
name_mapping <- setNames(map_id_name$gene_name, map_id_name$gene_id)
# name_mapping

In [ ]:
subset_and_melt <- function(matrix, gene_of_interest ) {
  matrix <- Cor_matrix[gene_of_interest, gene_of_interest] 
  rownames(matrix) <- name_mapping[rownames(matrix)]
  colnames(matrix) <- name_mapping[colnames(matrix)]
  matrix <- matrix%>% melt
  return(matrix)
}

In [ ]:
TF_of_interest <- read.csv('diffbind_results_2DM/unique_TFs.csv') %>% pull(1) 

In [ ]:
Cor_matrix <- subset_and_melt(Cor_matrix_all, TF_of_interest)
Syn_matrix <- subset_and_melt(Synergy_matrix_all, TF_of_interest)

In [ ]:
Cor_matrix %>% filter(value %>% abs > 0.98) %>% dplyr::rename( Cor_value = value) %>% 
    left_join(Syn_matrix %>% dplyr::rename( Syn_value = value), by = c("Var1", "Var2")) 

In [ ]:
Cor_Syn_values <- Cor_matrix %>% filter(value %>% abs > 0.98) %>% dplyr::rename( Cor_value = value) %>% 
    left_join(Syn_matrix %>% dplyr::rename( Syn_value = value), by = c("Var1", "Var2")) 
Cor_Syn_values

In [ ]:
options(repr.plot.width= 8, repr.plot.height= 8, repr.plot.res = 300)

# Creating the heatmap with ggplot2
#fill is Cor, number is Synergy
plot <- ggplot(data = Cor_Syn_values, aes(x=Var1, y=Var2, fill=Cor_value)) +
  geom_tile(aes(size = abs(Cor_value))) + # Size by the absolute value of synergy level
  scale_fill_gradient2(low = "#5793B3", high = "#CA5351", mid = "white", 
                       midpoint = 0, limit = c(-1,1), space = "Lab", 
                       name="Accessibility\nCorrelation Level") +
  geom_text(aes(label = round(Syn_value, 2)), size = 4) + # Add text labels
  theme_minimal() + theme_Arial_8() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1),
        legend.position = "bottom",
        panel.grid.major = element_blank(),
        panel.grid.minor = element_blank(),
        panel.border = element_blank(),
        axis.ticks = element_blank()) + 
  labs(title = "Correlation and synergy between pairs of TFs") +
     scale_x_discrete( expand = c(0, 0)) + scale_y_discrete( expand = c(0, 0)) +

  labs(x = "", y = "", title = "Correlation and synergy between pairs of TF motifs")

plot

In [ ]:
#generate a random number between 0 and 10000
random_number <- runif(1, 0, 10000)
round(random_number)